## Merge CRSP and Compustat
The purpose of this file is to demnostrate how to merge CRSP and Compustat to map PERMNO-GVKEY pairs without CRSP-Compustat Merge table (CCM).
However, you should keep in mind that CCM will always give you more complete match than using CUSIP or NCUSIP. 

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandasql import sqldf

pysql = lambda q: sqldf(q,globals())
pd.options.display.width = 180

### Read required data
We need the following data:
1. CRSP monthly stock file
2. CRSP stock name history file
3. Compustat stock name history file

For second and third dataset, you can retrieve via WRDS cloud. The source is **crsp.msenames** and **comp.security**.

In [2]:
# Read CRSP stock history file
msenames = pd.read_csv('/users/ml/data/wrds/raw/msenames.txt',sep='\t')
msenames.columns = msenames.columns.map(str.lower)
msenames_1 = msenames[['permno','cusip','ncusip','namedt','nameendt','comnam','ticker','shrcd','exchcd']].copy()
msenames_1 = msenames_1.sort_values(['permno','namedt']).reset_index(drop=True)

# Read Compustat stock history file
security = pd.read_csv('/users/ml/data/wrds/raw/security.txt',sep='\t')
security.columns = security.columns.map(str.lower)
# Convert Compustat 9-digit CUSIP to 8-digit CUSIP
security['cusip'] = security['cusip'].apply(lambda x: str(x)[:8])
security['ncusip'] = security['cusip']
# Remove missing CUSIP and keep stocks listing in US stock exchanges (this can remove dual listing in US and Canada)
security_1 = security[(security['cusip'].notnull())&(security['cusip']!='nan')&(security['excntry']=='USA')].copy()
security_1 = security_1.sort_values('gvkey').reset_index(drop=True)

# Read CRSP monthly file
# This file already deletes duplicates and keeps only stocks listing in NYSE/NASDAQ/AMEX
msf = (pd.read_hdf('/users/ml/data/clean/wrds/msf.h5','msf',
                   columns=['permno','permco','cusip','date','ncusip','comnam','ticker','shrcd','exchcd']))
# Keep common shares and stock with NCUSIP
# Stocks without NCUSIP in their whole life cannot match with Compustat,
# because such stocks have no official CUSIP and the CUSIP is a dummy CUSIP assigned by CRSP
msf_1 = msf[msf['shrcd'].isin([10,11])&(msf['ncusip'].notnull())].copy()

### PERMNO-GVKEY link using current CUSIP
The drawback of this way is that the merge is not accurate if you do not download the two databases at the same time.
For example, your CRSP is downloaded 2 years ago and your Compustat is downloaded recently. Some CUSIPs may already change and be updated then you can never match them by the latest CUSIP.

In [3]:
crsp_cusip = msf_1.drop_duplicates(subset='cusip')[['permno','cusip','ncusip','comnam','ticker']]
crsp_cusip = crsp_cusip.sort_values('cusip').reset_index(drop=True)
link_cc_all = crsp_cusip.merge(security_1[['gvkey','cusip']],how='left',on='cusip')
link_cc = crsp_cusip.merge(security_1[['gvkey','cusip']],how='inner',on='cusip')
print('Number of matched PERMNO-GVKEY: {}'.format(len(link_cc)))
print('Percent of match: {:3.1%}'.format(len(link_cc)/len(link_cc_all)))

Number of matched PERMNO-GVKEY: 19260
Percent of match: 82.4%


### PERMNO-GVKEY link using NCUSIP
Here using NCUSIP means using CRSP NCUSIP to merge Compustat CUSIP (Compustat does not keep historical CUSIP).

In [4]:
# Keep stocks with NCUSIP respect to correct date range
# Observations may exist even after the end of name
ncusip_track = pysql("""select a.permno,a.cusip,a.ncusip,a.date,a.comnam,a.ticker,b.namedt,b.nameendt
                   from msf_1 a inner join msenames_1 b 
                   on a.permno=b.permno and a.date>=b.namedt and a.date<=b.nameendt;""")

permno_gvkey = msenames_1.merge(security_1[['gvkey','ncusip']],how='inner',on='ncusip')
permno_gvkey = permno_gvkey.sort_values(['permno','ncusip']).reset_index(drop=True)
link_nc_all = ncusip_track[['permno','cusip','ncusip']].merge(permno_gvkey[['permno','gvkey']],how='left',on='permno')
link_nc_all = link_nc_all.drop_duplicates(subset='permno')
link_nc = ncusip_track[['permno','cusip','ncusip']].merge(permno_gvkey[['permno','gvkey']],how='inner',on='permno')
link_nc = link_nc.drop_duplicates(subset=['permno','gvkey'])
link_nc = link_nc.sort_values(['permno','gvkey']).reset_index(drop=True)
print('Number of matched PERMNO-GVKEY: {}'.format(len(link_nc)))
print('Percent of match: {:3.1%}'.format(len(link_nc)/len(link_nc_all)))

Number of matched PERMNO-GVKEY: 19526
Percent of match: 83.6%


### Some checks
Overall, there are more matched pairs using NCUSIP than CUSIP, although the difference is small. The missing pairs using CUSIP method are due to two secnarios:
1. Never match with Compustat: there is no match between CRSP CUSIP and Compustat CUSIP.
2. Only match with Compustat in a specific period: one PERMNO may have multiple GVKEYs.

#### Never match case

In [5]:
# There is no matched GVKEY for PERMNO 92321 using CUSIP method
link_cc[link_cc['permno']==92321]

,permno,cusip,ncusip,comnam,ticker,gvkey


In [6]:
# There is a matched GVKEY for PERMNO 92321 using NCUSIP method
link_nc[link_nc['permno']==92321]

,permno,cusip,ncusip,gvkey
19087,92321,87670L10,87670110,10346


In [7]:
msenames_1[msenames_1['permno']==92321]

,permno,cusip,ncusip,namedt,nameendt,comnam,ticker,shrcd,exchcd
92666,92321,87670L10,87670110,19800604,19821031,TAURUS OIL CORP,TAUR,11,3
92667,92321,87670L10,87670110,19821101,19830113,TAURUS OIL CORP,TAUR,11,3
92668,92321,87670L10,87670110,19830114,19830516,TAURUS OIL CORP,TAUR,11,3
92669,92321,87670L10,87670110,19830517,19850120,TAURUS OIL CORP,NaN,11,0
92670,92321,87670L10,87670L10,19850121,19900730,TAURUS PETROLEUM INC,TAUR,11,3


In [8]:
# There is no this CUSIP in Compustat, this is why there is no matched GVKEY using CUSIP 
security_1[security_1['cusip']=='87670L10'][['gvkey','cusip','iid','tic','tpci']]

,gvkey,cusip,iid,tic,tpci


In [9]:
# Using the matched PERMNO-GVKEY pair, we can check if the CUSIP 
security_1[security_1['gvkey']==10346][['gvkey','cusip','iid','tic','tpci']]

,gvkey,cusip,iid,tic,tpci
10035,10346,87670110,01,TAURC,0


#### Incomplete match case

In [10]:
link_cc[link_cc['permno']==64822]

,permno,cusip,ncusip,comnam,ticker,gvkey
19120,64822,98872F10,74721510,PYRAMID OIL CO,PYOL,28788


In [11]:
link_nc[link_nc['permno']==64822]

,permno,cusip,ncusip,gvkey
8645,64822,98872F10,74721510,8838
8646,64822,98872F10,74721510,21732
8647,64822,98872F10,74721510,28788


In [12]:
msenames_1[msenames_1['permno']==64822]

,permno,cusip,ncusip,namedt,nameendt,comnam,ticker,shrcd,exchcd
41897,64822,98872F10,74721510,19760719,19920707,PYRAMID OIL CO,PYOL,10,3
41898,64822,98872F10,74721510,19920708,20060820,PYRAMID OIL CO,NaN,11,0
41899,64822,98872F10,74721510,20060821,20140910,PYRAMID OIL CO,PDO,11,2
41900,64822,98872F10,98872E10,20140911,20161026,YUMA ENERGY INC,YUMA,11,2
41901,64822,98872F10,98872F10,20161027,20161230,YUMA ENERGY INC NEW,YUMA,11,2


In [13]:
# Match between 1976 and 2014
security_1[security_1['gvkey']==8838][['gvkey','cusip','iid','tic','tpci']]

,gvkey,cusip,iid,tic,tpci
8421,8838,74721510,01,PDO,0


In [14]:
# Match between 2014 and 2016
security_1[security_1['gvkey']==21732][['gvkey','cusip','iid','tic','tpci']]

,gvkey,cusip,iid,tic,tpci
19018,21732,98872E10,01,YUMA.1,0
19019,21732,98872E20,02,YUMA.PA,G


In [15]:
# Match bewteen 2016 and most recent
security_1[security_1['gvkey']==28788][['gvkey','cusip','iid','tic','tpci']]

,gvkey,cusip,iid,tic,tpci
23868,28788,98872F10,01,YUMA,0
